## Processing acc data (Lili's dataset)

In [1]:
import math
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"participant0"+str(7)+"/acc.csv"
data = pd.read_csv(path)
data.head()

,id,p_id,e_id,itr_id,itr_type,hand,clt_id,x,y,z,timestamp
0,187985,7,6,102,0,0,-1,-0.385864,-0.085861,0.916092,2021-08-05 13:19:00.518
1,187986,7,6,102,0,1,-1,0.449539,-0.049988,0.928543,2021-08-05 13:19:00.522
2,187987,7,6,102,0,1,-1,0.443680,-0.043152,0.918777,2021-08-05 13:19:00.541
3,187988,7,6,102,0,0,-1,-0.382446,-0.072189,0.901932,2021-08-05 13:19:00.544
4,187989,7,6,102,0,1,-1,0.453934,-0.041199,0.925613,2021-08-05 13:19:00.562


### Step 1 - Only keep the interactions for the 5 properties

In [3]:
prop_id = [3,4,5,6,7,8]

for participant in range(1,9):
    print("Participant: ", participant)

    path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc.csv"

    data = pd.read_csv(path)

    props = data[data['itr_type'].isin(prop_id)] 

    write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc_prop.csv"
    props.to_csv(write_path, index = None)

Participant:  1
Participant:  2
Participant:  3
Participant:  4
Participant:  5
Participant:  6
Participant:  7
Participant:  8


In [4]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(8)+"/acc_prop.csv"
data = pd.read_csv(path)
data.head()


,id,p_id,e_id,itr_id,itr_type,hand,clt_id,x,y,z,timestamp
0,107434,8,4,55,3,0,8,-0.795776,0.143311,0.697784,2021-08-10 03:11:23.426
1,107435,8,4,55,3,1,8,1.032761,0.382584,0.198761,2021-08-10 03:11:23.438
2,107436,8,4,55,3,0,8,-0.765991,0.177490,0.669464,2021-08-10 03:11:23.451
3,107437,8,4,55,3,1,8,1.014206,0.383560,0.230499,2021-08-10 03:11:23.462
4,107438,8,4,55,3,1,8,1.066940,0.196060,0.205109,2021-08-10 03:11:23.479


### Step 2 - Sort the files in temporal order

In [5]:
for participant in range(1,9):
    print("Participant: ", participant)

    path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc_prop.csv"
    data = pd.read_csv(path)

    data['timestamp']=pd.to_datetime(data['timestamp'])

    data.sort_values(by=['timestamp'], inplace=True)

    itr_id = data.itr_id.unique()

    write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc_prop_sorted.csv"
    data.to_csv(write_path,index=False)

Participant:  1
Participant:  2
Participant:  3
Participant:  4
Participant:  5
Participant:  6
Participant:  7
Participant:  8


### Step 3 - Compute accelleration, velocity and jerkiness for x, y and z directions from the (linear) accelerometer data we have

In [6]:
labels = ['id','p_id','e_id','itr_id','itr_type','clt_id','hand','Ax','Ay','Az','Vx','Vy','Vz','Jx','Jy','Jz','timestamp']

for participant in range(1,9):
    print("Participant: ", participant)

    path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc_prop_sorted.csv"

    data = pd.read_csv(path)


    data['timestamp']=pd.to_datetime(data['timestamp'])
    #data['timestamp']= data['timestamp'].to_timestamp()

    #data['x'] = data['x']/65536.0
    #data['y'] = data['y']/65536.0
    #data['z'] = data['z']/65536.0

    hand0 = data[data['hand'] == 0]
    hand1 = data[data['hand'] == 1]


    speed0x = [0]*len(hand0)
    speed1x = [0]*len(hand1)

    speed0y = [0]*len(hand0)
    speed1y = [0]*len(hand1)

    speed0z = [0]*len(hand0)
    speed1z = [0]*len(hand1)

    jerk0x = [0]*len(hand0)
    jerk1x = [0]*len(hand1)

    jerk0y = [0]*len(hand0)
    jerk1y = [0]*len(hand1)

    jerk0z = [0]*len(hand0)
    jerk1z = [0]*len(hand1)

    for i in range(1,len(hand0)):
        if hand0.iloc[i].at['itr_id'] != hand0.iloc[i-1].at['itr_id']:
            speed0x[i] = 0
            speed0y[i] = 0
            speed0z[i] = 0
            jerk0x[i] = 0
            jerk0y[i] = 0
            jerk0z[i] = 0
        else:    
            time1 = hand0.iloc[i-1]['timestamp']
            time2 = hand0.iloc[i]['timestamp']
            vel_x = (hand0['x'].iloc[i] - hand0['x'].iloc[i-1])*(time2-time1).total_seconds()
            vel_y = (hand0['y'].iloc[i] - hand0['y'].iloc[i-1])*(time2-time1).total_seconds()
            vel_z = (hand0['z'].iloc[i] - hand0['z'].iloc[i-1])*(time2-time1).total_seconds()
            jerk_x = (hand0['x'].iloc[i] - hand0['x'].iloc[i-1])/(time2-time1).total_seconds()
            jerk_y = (hand0['y'].iloc[i] - hand0['y'].iloc[i-1])/(time2-time1).total_seconds()
            jerk_z = (hand0['z'].iloc[i] - hand0['z'].iloc[i-1])/(time2-time1).total_seconds()
            speed0x[i]=vel_x
            speed0y[i]=vel_y
            speed0z[i]=vel_z
            jerk0x[i]=jerk_x
            jerk0y[i]=jerk_y
            jerk0z[i]=jerk_z



    for i in range(1,len(hand1)):
        if hand1.iloc[i].at['itr_id'] != hand1.iloc[i-1].at['itr_id']:
            speed1x[i] = 0
            speed1y[i] = 0
            speed1z[i] = 0
            jerk1x[i] = 0
            jerk1y[i] = 0
            jerk1z[i] = 0
        else:
            time1 = hand1.iloc[i-1]['timestamp']
            time2 = hand1.iloc[i]['timestamp']
            vel_x = (hand1['x'].iloc[i] - hand1['x'].iloc[i-1])*(time2-time1).total_seconds()
            vel_y = (hand1['y'].iloc[i] - hand1['y'].iloc[i-1])*(time2-time1).total_seconds()
            vel_z = (hand1['z'].iloc[i] - hand1['z'].iloc[i-1])*(time2-time1).total_seconds()
            jerk_x = (hand1['x'].iloc[i] - hand1['x'].iloc[i-1])/(time2-time1).total_seconds()
            jerk_y = (hand1['y'].iloc[i] - hand1['y'].iloc[i-1])/(time2-time1).total_seconds()
            jerk_z = (hand1['z'].iloc[i] - hand1['z'].iloc[i-1])/(time2-time1).total_seconds()
            speed1x[i]=vel_x
            speed1y[i]=vel_y
            speed1z[i]=vel_z
            jerk1x[i]=jerk_x
            jerk1y[i]=jerk_y
            jerk1z[i]=jerk_z

    tot0 = pd.DataFrame(hand0, columns=labels)
    tot0['Ax'] = hand0['x']
    tot0['Ay'] = hand0['y']
    tot0['Az'] = hand0['z']
    tot0['Vx'] = speed0x
    tot0['Vy'] = speed0y
    tot0['Vz'] = speed0z
    tot0['Jx'] = jerk0x
    tot0['Jy'] = jerk0y
    tot0['Jz'] = jerk0z

    tot1 = pd.DataFrame(hand1, columns=labels)
    tot1['Ax'] = hand1['x']
    tot1['Ay'] = hand1['y']
    tot1['Az'] = hand1['z']
    tot1['Vx'] = speed1x
    tot1['Vy'] = speed1y
    tot1['Vz'] = speed1z
    tot1['Jx'] = jerk1x
    tot1['Jy'] = jerk1y
    tot1['Jz'] = jerk1z
    #tot = pd.DataFrame([hand0['id'],hand0['p_id'],hand0['e_id'],hand0['itr_id'],hand0['itr_type'],hand0['clt_id'],hand0['hand'],
    #    hand0['x'],hand0['y'],hand0['z'],speed0x,speed0y,speed0z,hand0['timestamp']],columns=labels)

    tot3 = pd.concat([tot0,tot1])
    write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(participant)+"/acc_speeds.csv"
    tot3.to_csv(write_path, index = None)


Participant:  1
Participant:  2
Participant:  3
Participant:  4
Participant:  5
Participant:  6
Participant:  7
Participant:  8


In [7]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(6)+"/acc_speeds.csv"
#print(path)
data = pd.read_csv(path)
data.shape


(71839, 17)

### Step 4 - Combine the datasets

In [8]:
path1 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(1)+"/acc_speeds.csv"
data1 = pd.read_csv(path1)

path2 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(2)+"/acc_speeds.csv"
data2 = pd.read_csv(path2)

path3 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(3)+"/acc_speeds.csv"
data3 = pd.read_csv(path3)

path4 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(4)+"/acc_speeds.csv"
data4 = pd.read_csv(path4)

path5 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(5)+"/acc_speeds.csv"
data5 = pd.read_csv(path5)

path6 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(6)+"/acc_speeds.csv"
data6 = pd.read_csv(path6)

path7 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(7)+"/acc_speeds.csv"
data7 = pd.read_csv(path7)

path8 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/"+"Participant0"+str(8)+"/acc_speeds.csv"
data8 = pd.read_csv(path8)


In [9]:
acc_combined_1 = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8], axis=0)

In [10]:
write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/Lili_Experiment/acc_combined_lili.csv"
acc_combined_1.to_csv(write_path, index = None)

